In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
from tensorflow import keras
from tensorflow.keras import  layers
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, UpSampling2D, Concatenate


In [2]:
import sys
sys.path.insert(0, '../lib/')
from help_functions import *

#function to obtain data for training/testing (validation)
# from extract_patches import get_data_training
sys.path.insert(0, '../lib/networks/')
# from fully_conv import get_fully_conv_unet as net_
# from fully_conv import attend
from preprocessing import preprocessing

In [3]:
print(tf.keras.__version__)

2.1.6-tf


In [4]:
exp_path = 'experiments/'
exp_name = 'exp1'

In [5]:
train_img, label_img = preprocessing(exp_path+exp_name)


train images/masks shape:
(20, 1, 565, 565)
train images range (min-max): 0.0 - 1.0
train masks are within 0-1

patches per full image: 1500

train PATCHES images/masks shape:
(30000, 1, 128, 128)
train PATCHES images range (min-max): 0.00784313725490196 - 1.0
(30000, 1, 128, 128)
128 128 1
......DONE......


In [6]:
train_img.shape, label_img.shape

((30000, 1, 128, 128), (30000, 16384, 2))

In [7]:
def conv_block(chan_out):
    chan = 'channels_first'
    return keras.Sequential([        
        Conv2D(chan_out, (3, 3), activation='relu', padding='same',data_format=chan),
        Conv2D(chan_out, (3, 3), activation='relu', padding='same',data_format=chan)
    ])

def up_block(chan_out,input_, block):
    chan = 'channels_first'
    upsample = layers.UpSampling2D((2,2),data_format=chan)(input_)
    upconv = layers.Conv2DTranspose(chan_out,(3,3),padding='same',data_format=chan)(upsample)
    concat = Concatenate(axis=1)([block, upconv])
    return conv_block(chan_out)(concat)


def get_unet(n_ch,patch_height, patch_width, debug=False):
    chan = 'channels_first'
    inputs = keras.Input(shape=(n_ch, patch_height, patch_width))

    if debug : print(inputs.shape)
        
    #+++++++++++++++++++++++++++++++++++++++++++++++++++    
    #EDIT HERE 
    #+++++++++++++++++++++++++++++++++++++++++++++++++++
    #encode
    block1 = conv_block(64)(inputs) # ------------->to down1
    maxPool1 = MaxPooling2D((2, 2), data_format=chan)(block1)
    
    block2 = conv_block(128)(maxPool1) #----------->to down2
    maxPool2 = MaxPooling2D((2, 2), data_format=chan)(block2)
   
    block3 = conv_block(256)(maxPool2) #----------->to down3
    maxPool3 = MaxPooling2D((2, 2), data_format=chan)(block3)
    
    block4 = conv_block(512)(maxPool3) #----------->to down4
    maxPool4 = MaxPooling2D((2, 2), data_format=chan)(block4)
   
    #bottom
    block5 = conv_block(1024)(maxPool4) # ----------> bottom
    
    #decode
    down4 = up_block(512,block5,block4)
    down3 = up_block(256, down4,block3)
    down2 = up_block(128, down3,block2)
    down1 = up_block(64, down2,block1)
    down1 = Dropout(.5)(down1)
    
    if debug : print(down1.shape)
    
    #+++++++++++++++++++++++++++++++++++++++++++++++++++
    
    conv1x1 = Conv2D(2, (1, 1), activation='relu',padding='same',data_format=chan)(down1)

    conv1x1 = layers.Reshape((2,patch_height*patch_width))(conv1x1)
    
    conv1x1 = layers.Permute((2,1))(conv1x1)

    out = layers.Activation('softmax')(conv1x1)
       
    model = keras.models.Model(inputs=inputs, outputs=out)
    print(model.summary())

    opt = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [8]:
model = get_unet(1,128,128)

#save weights
sav_path = '../'+exp_path+exp_name+'/'+exp_name

checkpoints = keras.callbacks.ModelCheckpoint(filepath=sav_path+'_best_w.h5', 
                                            monitor='val_loss',verbose=2,
                                            save_best_only=True)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 128, 128)  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 64, 128, 128) 37568       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 64)   0           sequential[0][0]                 
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 128, 64, 64)  221440      max_pooling2d[0][0]              
__________________________________________________________________________________________________
max_poolin

In [9]:
history = model.fit(train_img, label_img, batch_size=32,epochs=10,
                    validation_split = 0.1, 
                    shuffle=True, verbose=2, 
                    callbacks=[checkpoints])


Train on 27000 samples, validate on 3000 samples
Epoch 1/10

Epoch 00001: val_loss improved from inf to 0.24968, saving model to ../experiments/exp1/exp1_best_w.h5
 - 256s - loss: 0.3773 - acc: 0.8733 - val_loss: 0.2497 - val_acc: 0.9141
Epoch 2/10

Epoch 00002: val_loss improved from 0.24968 to 0.16445, saving model to ../experiments/exp1/exp1_best_w.h5
 - 248s - loss: 0.2115 - acc: 0.9241 - val_loss: 0.1645 - val_acc: 0.9371
Epoch 3/10

Epoch 00003: val_loss improved from 0.16445 to 0.14928, saving model to ../experiments/exp1/exp1_best_w.h5
 - 248s - loss: 0.1654 - acc: 0.9389 - val_loss: 0.1493 - val_acc: 0.9431
Epoch 4/10


KeyboardInterrupt: 